# Speedup
Now that you have an eval and things are looking good, to speed things up in production.
Unfortunately there's no free lunch though. 
Let's change some inference parameters and see if we can get a speedup,
but use our evals to understand what the tradeoff is.

## Steps
* Run inference but this time focus on engineering metrics, in our case we care about clock time
* Measure 5 responses using the an fp16 model, then repeat with a fp2 model
* Before running evals, what is different about these models?
  * Hints: Google quantization, look at the model size on Ollama, think about what is expensive during inference.
* After runing evals
  * What do you notice is different?
  * What about the quality of the responses? How does that change?



## Temporary References
* Enable KV Cache quantization https://github.com/ollama/ollama/blob/main/docs/faq.md#how-can-i-set-the-quantization-type-for-the-kv-cache
* https://github.com/ollama/ollama/blob/main/docs/faq.md#how-can-i-enable-flash-attention
https://github.com/ollama/ollama/blob/main/docs/faq.md#how-can-i-set-the-quantization-type-for-the-kv-cache

In [43]:
from ollama import chat
from ollama import ChatResponse
import time
import os

In [44]:
article = """
\n\n**A Journey Through Time: The History and Achievements of North Melbourne Kangaroos**\n\nThe North Melbourne Kangaroos, affectionately known as the Magpies prior to their 1983 rebranding, are a storied team with a rich tapestry of history. Nestled in Ballarat, Victoria, these kangaroos have been the pride of the region since their inception in 1924. This article delves into their remarkable journey, from their early days as a minor league side to becoming a significant force in Australian rugby union.\n\n**The Founding of the Kangaroos**\n\nNorth Melbourne Kangaroos were born on November 6, 1924, when the North Melbourne Rugby Union Club was established. The club\'s name change to Kangaroos occurred after their 1983 merger with Torquay United, a move that reflected their transition from being based in Melbourne to Ballarat.\n\n**A Legacy of Excellence**\n\nThe Kangaroos\' history is marked by excellence and resilience. Over the decades, they\'ve claimed multiple State Titles, showcasing their dominance on the field. Notable players such as David Nasmith and Adam Droms have been luminaries of the team, with Nasmith holding the distinction of being one of the youngest players to grace the field in 1958 and still active today.\n\n**Current Stance in the League**\n\nSince their move to Ballarat in 2019, the North Melbourne Kangaroos have become a prominent side in the Victorian top-tier competition. Their transition was met with both anticipation and nostalgia from their old fans, who cherished the memories of playing at Belmore Ground, the club\'s historic home.\n\n**Community Connection**\n\nThe kangaroos are deeply rooted in the Ballarat community. Their branding includes the iconic "Kangaroo" logo, symbolizing their connection to the local wildlife. The team has also embraced community involvement, with players and staff engaging actively in local initiatives, fostering a strong bond between the club and its surroundings.\n\n**Engaging with Fans**\n\nIn keeping with rugby union traditions, the Kangaroos embrace close-knit relationships with their fans. Whether through match days or annual events, the team consistently demonstrates a commitment to community spirit, further cementing its identity as a local institution.\n\n**Looking Ahead**\n\nThe future of North Melbourne Kangaroos looks promising. With a focus on developing young talent and maintaining high standards, the club is poised for continued success. Fans can expect an exciting season ahead, with the kangaroos aiming to uphold their legacy while exploring new horizons in the league.\n\nIn conclusion, the North Melbourne Kangaroos represent more than just a rugby team; they are a symbol of pride for Ballarat and a testament to the enduring spirit of sportsmanship. As they continue to navigate the challenges and opportunities of modern rugby union, one can look forward to many thrilling matches and memorable moments in their storied history.'
"""

In [49]:
model = 'gemma2:2b'

def single_turn_with_time(prompt, model):
    start_time = time.time()
    response: ChatResponse = chat(model=model, messages=[
      {
        'role': 'user',
        'content': prompt,
      },
    ])
    end_time = time.time()
    
    total_time = end_time - start_time
    return response.message.content, total_time


prompt = f"Is this about an australian or american team?, print no other words: {article}?"
single_turn_with_time(prompt, model)

('Australian \n', 0.13673710823059082)

In [46]:
# os.environ["OLLAMA_KV_CACHE_TYPE"] = "FP16"
# os.environ["OLLAMA_FLASH_ATTENTION"] = "0"

model = 'gemma2:2b-text-fp16'

for i in range(5):
    prompt = f"{i} Is this about an australian or american team?, print no other words: {article}?"
    print(single_turn_with_time(prompt, model=model)[1])

gemma2:2b-text-fp16
13.946565866470337
gemma2:2b-text-fp16
0.4636821746826172
gemma2:2b-text-fp16
0.46567702293395996
gemma2:2b-text-fp16
19.043325901031494
gemma2:2b-text-fp16
0.7935211658477783


In [50]:
for i in range(5):
    prompt = f"{i} Is this about an australian or american team?, print no other words: {article}?"
    response, wall_time = single_turn_with_time(prompt, model=model)
    character_per_second = len(response) / wall_time
    print(character_per_second, response)

23.573416164660987 Australian 

23.687432141470474 Australian 

23.732119214041337 Australian 

23.657115139117895 Australian 

23.660329090157806 Australian 



## Quantized model
Let's try the quantized model. See if there's anything you notice

### Key Points for students
* Notice that the tokens per second is much faster
* Also notice though that the quality is much worse. The model is outputting all sorts of weird characters and is barely usable
* Is this tradeoff worth it? No

In [40]:
#os.environ["OLLAMA_KV_CACHE_TYPE"] = "q4_0"

model = 'gemma2:2b-instruct-q2_K'

for i in range(5):
    prompt = f"{i} Is this about an australian or american team?, print no other words: {article}?"
    print(single_turn_with_time(prompt, model=model)[1])

gemma2:2b-instruct-q2_K
0.6870639324188232
gemma2:2b-instruct-q2_K
0.5511720180511475
gemma2:2b-instruct-q2_K
0.5475988388061523
gemma2:2b-instruct-q2_K
0.5449249744415283
gemma2:2b-instruct-q2_K
0.5457031726837158


In [52]:
model = 'gemma2:2b-instruct-q2_K'

for i in range(5):
    prompt = f"{i} Is this about an australian or american team?, print no other words: {article}?"
    response, wall_time = single_turn_with_time(prompt, model=model)
    character_per_second = len(response) / wall_time
    print(character_per_second, response)

128.2925066907546 Australian 


 
 
 
 

*   
 


 
 




 
 


 **Please note:** This text is a bit more information about the team itself.  I am still learning how to be helpful! 
 
 
 
 


18.211589964117866 American 

109.38831121570149 American 


Please let me know if you'd like further information about this team!  🏈 

21.879891025799864 Australian 

100.45768390929733 American 


I hope this helps!  Let me know if you have any other questions.

